# Кластеры. Когда нужно больше точек

Используем набор данных `moscow_malls_info.csv` с информацией о торговых центрах Москвы.

Датасет хранит следующие данные:

- `name` — название торгового центра;
- `address` — адрес торгового центра;
- `hours` — часы работы торгового центра;
- `lat` — широта точки, в которой находится объект;
- `lng` — долгота точки, в которой находится объект;
- `rating` — рейтинг центра на Яндекс Картах;
- `district` — район Москвы, в котором находится объект.

Исследуем набор данных:

In [1]:
import pandas as pd

df = pd.read_csv('/datasets/moscow_malls_info.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      786 non-null    object 
 1   address   786 non-null    object 
 2   hours     685 non-null    object 
 3   lat       786 non-null    float64
 4   lng       786 non-null    float64
 5   rating    786 non-null    float64
 6   district  764 non-null    object 
dtypes: float64(3), object(4)
memory usage: 43.1+ KB


Датасет хранит информацию о 786 торговых центрах.

Посмотрим на первые строки:

In [2]:
df.head()

,name,address,hours,lat,lng,rating,district
0,Капитолий,"Москва, Правобережная улица, 1Б","ежедневно, 10:00–22:00",55.880803,37.449824,4.7,Северный административный округ
1,Dexter,"Москва, МКАД, 78-й километр, 14, корп. 1","ежедневно, 10:00–21:00",55.893082,37.501263,4.3,Северный административный округ
2,РИО,"Москва, Дмитровское шоссе, 163А, корп. 1","ежедневно, 10:00–22:00",55.909388,37.539546,4.8,Северо-Восточный административный округ
3,Экстрим,"Москва, Смольная улица, 63Б","ежедневно, 10:00–21:00",55.869937,37.469915,4.7,Северный административный округ
4,Discovery,"Москва, улица Дыбенко, 7/1","ежедневно, 09:00–21:00",55.878662,37.479251,4.3,Северный административный округ


Отметим первые 5 заведений на карте:

In [3]:
# подключаем библиотеку
from folium import Marker, Map

# moscow_lat - широта, moscow_lng – долгота
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его на карту
def create_marker(row):
    Marker([row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}"
    ).add_to(m)

# применяем функцию для создания маркера к первым 5 записям датафрейма
df[:5].apply(create_marker, axis=1)
    
# выводим карту
m

А теперь отметим сразу все здания:

In [4]:
# подключаем библиотеку
from folium import Marker

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его на карту
def create_marker(row):
    Marker([row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}"
    ).add_to(m)

# применяем функцию для создания маркера ко всем строкам датафрейма
df.apply(create_marker, axis=1)
    
# выводим карту
m

Здесь могут возникнуть проблемы:

- Карта может долго отрисовываться, или рендериться (от англ. render, «выводить, отображать»), на слабых по производительности компьютерах или серверах;
- Если на карте много точек, при уменьшении масштаба в ней будет сложно ориентироваться.
- Из-за высокой нагрузки карта может зависать при передвижении по ней.

Если отобразить на карте несколько тысяч точек, эти проблемы станут особенно заметны. Помогут кластеры — с их помощью можно поместить на карту много маркеров одновременно. 

Отметим все торговые центры на карте с помощью кластеров:

In [5]:
# импортируем карту и маркер
from folium import Map, Marker
# импортируем кластер
from folium.plugins import MarkerCluster

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
df.apply(create_clusters, axis=1)

# выводим карту
m

Попробуйте увеличить масштаб с помощью колёсика мышки или кнопок на карте. Вы увидите, что кластеры автоматически перегруппировываются: при достаточном приближении из кластеров начнут «выпадать» отдельные маркеры с торговыми центрами. Всплывающие окна и подсказки для них реализованы точно так же, как и для отдельных маркеров.